### ■ Image Files resize (32x32)

In [ ]:
import os
import cv2 # 이미지나 영상을 처리하는 모듈

path = ["E:\data10\classification\dogs"] 
# E:\data10\classification\dogs 
# E:\data10\classification\hedgedogs

j = 0
for k in path:
    file_list = os.listdir(k)
    file_name = []
    for i in file_list:
        if  i not in ['resized_2.jpg','resized_3.jpg','resized_4.jpg']:
            file_name.append(int( i[:-4] ))
    file_name.sort()
    file_list = [k + '\\' + str(i) + '.jpg' for i in file_name]
    
    for i in file_list:
        img = cv2.imread(i)
        width, height = img.shape[:2]
        resize = cv2.resize(img, (int(32), int(32)), interpolation=cv2.INTER_CUBIC) # 현재 32 x 32로 resize
        cv2.imwrite("d:\\resize\\" + str(j + 1) + '.jpg', resize)
        j += 1

### ■ Create Labels

In [1]:
path = 'E:\data10\classification\\train_label.csv'
file = open( path, 'w')
for  i  in  range(1, 1630):
    file.write( str(1) + '\n' )
for  i  in  range(1, 978):
    file.write( str(0) + '\n') 
file.close()

In [2]:
path = 'E:\data10\classification\\test_label.csv'
file = open( path, 'w')
for  i  in  range(1, 182):
    file.write( str(1) + '\n' )
for  i  in  range(1, 109):
     file.write( str(0) + '\n') 
file.close()

### ■ Create loader3.py 

 ■ image_load  
 ■ next_batch  
 ■ shuffle_batch  
 ■ label_load  

In [ ]:
import  csv
import  os
import  re
import  cv2
import  random
import  numpy  as  np

def  image_load(path):
    file_list = os.listdir(path)
    file_name = []
    for  i  in  file_list:
        a = int(  re.sub('[^0-9]', '', i )  ) # 숫자가 아닌것은 '' 로 처리 
        file_name.append(a) 
    file_name.sort()

    file_res = [] 
    for  j  in   file_name:
        file_res.append('%s\\%d.jpg' %(path,j)  )

    image = []
    for  k  in  file_res:
        img = cv2.imread(k)
        image.append(img)

    return  np.array(image)

def  label_load( path ):
    file = open(path)
    labeldata = csv.reader(file)
    labellist = []
    for  i   in  labeldata:
        labellist.append(i)

    label = np.array(labellist)
    label = label.astype(int)  # 숫자로 변환 
    label = np.eye(2)[label]
    label = label.reshape(-1,2) 
    return  label


def  shuffle_batch( data_list, label ):
    x = np.arange( len( data_list) )
    random.shuffle(x)
    data_list2 = data_list[x]
    label2 = label[x]
    return   data_list2, label2 


def  next_batch( data1, data2, init,  fina ):
    return  data1[ init : fina ],  data2[init : fina] 

In [7]:
import loader3

train_image='E:\\data10\\classification\\train_resize\\'
train_label= 'E:\\data10\\classification\\train_label.csv'
test_image='E:\\data10\\classification\\test_resize\\'
test_label='E:\\data10\\classification\\test_label.csv'

print(loader3.image_load(train_image).shape)
print(loader3.label_load(train_label).shape)
print(loader3.image_load(test_image).shape)
print(loader3.label_load(test_label).shape)

(2606, 32, 32, 3)
(2606, 2)
(289, 32, 32, 3)
(289, 2)


### ■ Train start (vgg9.ver)

In [ ]:
import os, cv2, random

# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

import loader3

NUM_CLASSES = 2

train_image='E:\\data10\\classification\\train_resize\\'
train_label= 'E:\\data10\\classification\\train_label.csv'
test_image='E:\\data10\\classification\\test_resize\\'
test_label='E:\\data10\\classification\\test_label.csv'

print(loader3.image_load(train_image).shape)
print(loader3.label_load(train_label).shape)
print(loader3.image_load(test_image).shape)
print(loader3.label_load(test_label).shape)

x_train = loader3.image_load(train_image)
y_train = loader3.label_load(train_label)
x_test = loader3.image_load(test_image)
y_test = loader3.label_load(test_label)


# 모델 만들기 
input_layer = Input((32,32,3))

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)
x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)


x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = MaxPooling2D(pool_size=2,strides=2)(x)



# 완전 연결 계층 
x = Flatten()(x)

x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(1024)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 6개, 완전 연결계층 2개 출력층 1개인 9층 신경망 

model.summary()

# 모델 훈련

model_exists = False


if model_exists:
    
    
    model.load_weights('E:\data10\classification\\resultvgg9.h5')
    
    CLASSES = np.array(['개', '고슴도치' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test [1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)
    
else:
        
    opt = Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train
              , y_train
              , batch_size=28
              , epochs=50
              , shuffle=True
               , validation_data=(x_test,y_test) )
    
    
    model.save_weights('E:\data10\classification\\resultvgg9.h5')
    
    model.evaluate(x_test, y_test, batch_size=28)
    
    CLASSES = np.array(['개', '고슴도치' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test[1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)

### ■ Another train.ver (vgg9)

In [1]:
from keras.datasets import cifar10
from keras.models import Sequential, save_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import numpy as np
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
import loader3

batch_size = 28
num_classes = 2
epochs = 100
 
train_image='E:\\data10\\classification\\train_resize\\'
train_label= 'E:\\data10\\classification\\train_label.csv'
test_image='E:\\data10\\classification\\test_resize\\'
test_label='E:\\data10\\classification\\test_label.csv'


x_train = loader3.image_load(train_image)
y_train = loader3.label_load(train_label)
x_test = loader3.image_load(test_image)
y_test = loader3.label_load(test_label)      


print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape[1:])

#(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# One hot Encoding



model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))
 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
 
hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), nb_epoch=epochs, batch_size=batch_size, verbose=2)
 
scores = model.evaluate(x_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
 
save_model(model, "E:\data10\classification\models\\result2.h5")

Using TensorFlow backend.


(2606, 32, 32, 3)
(2606, 2)
(289, 32, 32, 3)
(289, 2)
(32, 32, 3)


C:\anaconda3\envs\tf2.0-gpu\lib\site-packages\ipykernel_launcher.py:75: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 2606 samples, validate on 289 samples
Epoch 1/100
 - 4s - loss: 0.8223 - accuracy: 0.6431 - val_loss: 0.7344 - val_accuracy: 0.6263
Epoch 2/100
 - 1s - loss: 0.5682 - accuracy: 0.7452 - val_loss: 1.0176 - val_accuracy: 0.6263
Epoch 3/100
 - 1s - loss: 0.4775 - accuracy: 0.7893 - val_loss: 0.5556 - val_accuracy: 0.7163
Epoch 4/100
 - 1s - loss: 0.3787 - accuracy: 0.8304 - val_loss: 0.6027 - val_accuracy: 0.6574
Epoch 5/100
 - 1s - loss: 0.3548 - accuracy: 0.8496 - val_loss: 0.4599 - val_accuracy: 0.7751
Epoch 6/100
 - 1s - loss: 0.3263 - accuracy: 0.8661 - val_loss: 0.5025 - val_accuracy: 0.7924
Epoch 7/100
 - 1s - loss: 0.4123 - accuracy: 0.8074 - val_loss: 0.5024 - val_accuracy: 0.7716
Epoch 8/100
 - 1s - loss: 0.2984 - accuracy: 0.8711 - val_loss: 0.8088 - val_accuracy: 0.6817
Epoch 9/100
 - 1s - loss: 0.3520 - accuracy: 0.8565 - val_loss: 0.4986 - val_accuracy: 0.7578
Epoch 10/100
 - 1s - loss: 0.2651 - accuracy: 0.8872 - val_loss: 0.3744 - val_accuracy: 0.8270
Epoch 11/100

Epoch 87/100
 - 1s - loss: 0.0636 - accuracy: 0.9793 - val_loss: 0.4072 - val_accuracy: 0.8270
Epoch 88/100
 - 1s - loss: 0.0482 - accuracy: 0.9800 - val_loss: 0.4236 - val_accuracy: 0.8201
Epoch 89/100
 - 1s - loss: 0.0420 - accuracy: 0.9843 - val_loss: 0.3858 - val_accuracy: 0.8304
Epoch 90/100
 - 1s - loss: 0.0755 - accuracy: 0.9731 - val_loss: 0.3381 - val_accuracy: 0.8789
Epoch 91/100
 - 1s - loss: 0.0524 - accuracy: 0.9843 - val_loss: 0.4556 - val_accuracy: 0.8443
Epoch 92/100
 - 1s - loss: 0.0318 - accuracy: 0.9893 - val_loss: 0.4470 - val_accuracy: 0.8408
Epoch 93/100
 - 1s - loss: 0.0396 - accuracy: 0.9862 - val_loss: 0.5352 - val_accuracy: 0.8131
Epoch 94/100
 - 1s - loss: 0.0259 - accuracy: 0.9919 - val_loss: 0.3748 - val_accuracy: 0.8789
Epoch 95/100
 - 1s - loss: 0.0305 - accuracy: 0.9908 - val_loss: 0.7548 - val_accuracy: 0.7647
Epoch 96/100
 - 1s - loss: 0.2531 - accuracy: 0.9163 - val_loss: 0.5505 - val_accuracy: 0.7958
Epoch 97/100
 - 1s - loss: 0.0954 - accuracy: 0.96

## ■ Vgg16.ver test

In [2]:
import os, cv2, random

# 필요한 라이브러리 import 하는 코드 
import numpy as np

from keras.layers import Input, Flatten, Dense, Conv2D, BatchNormalization, LeakyReLU, Dropout, Activation, MaxPooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import keras.backend as K   # 백엔드가  텐서 플로우로 되어있어서 
                                     # 텐서 플로우 명령어 필요할 때 tf 대신에 
                                     # k 를 쓰겠다라는 의미 

import loader3

NUM_CLASSES = 2

train_image='E:\\data10\\classification\\train_resize\\'
train_label= 'E:\\data10\\classification\\train_label.csv'
test_image='E:\\data10\\classification\\test_resize\\'
test_label='E:\\data10\\classification\\test_label.csv'

print(loader3.image_load(train_image).shape)
print(loader3.label_load(train_label).shape)
print(loader3.image_load(test_image).shape)
print(loader3.label_load(test_label).shape)

x_train = loader3.image_load(train_image)
y_train = loader3.label_load(train_label)
x_test = loader3.image_load(test_image)
y_test = loader3.label_load(test_label)

# 모델 만들기
input_layer = Input((32,32,3))

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(input_layer)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 64, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 128, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 256, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = Conv2D(filters = 512, kernel_size = 3, strides = 1, padding = 'same')(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)

x = MaxPooling2D(pool_size = 2, strides = 2)(x)

# 완전 연결 계층
x = Flatten()(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(4096)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(rate = 0.5)(x)

x = Dense(NUM_CLASSES)(x)
output_layer = Activation('softmax')(x)

model = Model(input_layer, output_layer)

# convolution 층 5개, 완전 연결계층 3개인 8층 신경망
model.summary()

# 모델 훈련

model_exists = False


if model_exists:
    
    
    model.load_weights('E:\data10\classification\\resultvgg9.h5')
    
    CLASSES = np.array(['개', '고슴도치' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test [1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)
    
else:
        
    opt = Adam(lr=0.0005)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(x_train
              , y_train
              , batch_size=28
              , epochs=50
              , shuffle=True
               , validation_data=(x_test,y_test) )
    
    
    model.save_weights('E:\data10\classification\\resultvgg16.h5')
    
    model.evaluate(x_test, y_test, batch_size=28)
    
    CLASSES = np.array(['개', '고슴도치' ]) 
    preds = model.predict( x_test[1:2] )   
    preds_value = CLASSES[ np.argmax( preds, axis= -1 ) ] 
    actual_value = CLASSES[ np.argmax( y_test[1:2], axis = -1) ] 
    print ( preds_value)
    print ( actual_value)

(2606, 32, 32, 3)
(2606, 2)
(289, 32, 32, 3)
(289, 2)
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_16 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_17 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
leaky

2606/2606 [==============================] - 4s 2ms/step - loss: 0.6150 - accuracy: 0.7421 - val_loss: 0.8588 - val_accuracy: 0.7612
Epoch 4/50
2606/2606 [==============================] - 4s 2ms/step - loss: 0.5067 - accuracy: 0.7905 - val_loss: 0.5669 - val_accuracy: 0.7474
Epoch 5/50
2606/2606 [==============================] - 4s 2ms/step - loss: 0.5223 - accuracy: 0.7832 - val_loss: 0.5694 - val_accuracy: 0.8512
Epoch 6/50
2606/2606 [==============================] - 4s 2ms/step - loss: 0.3954 - accuracy: 0.8384 - val_loss: 1.0754 - val_accuracy: 0.6574
Epoch 7/50
2606/2606 [==============================] - 4s 2ms/step - loss: 0.3298 - accuracy: 0.8726 - val_loss: 0.7112 - val_accuracy: 0.7751
Epoch 8/50
2606/2606 [==============================] - 4s 2ms/step - loss: 0.2801 - accuracy: 0.8949 - val_loss: 0.6461 - val_accuracy: 0.7889
Epoch 9/50
2606/2606 [==============================] - 4s 2ms/step - loss: 0.2269 - accuracy: 0.9079 - val_loss: 0.7932 - val_accuracy: 0.7889
Epo